In [1]:
import math
import os.path as path
import numpy as np
import pandas as pd
from skimage.util.montage import montage2d
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.layers import average, Input, Concatenate
from extra_functions import *
from scipy import signal


def load_and_format(in_path):
    out_df = pd.read_json(in_path)
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    out_images = np.stack(out_images).squeeze()
    return out_df, out_images

dir_path = path.abspath(path.join('__file__',"../.."))
train_path = dir_path + "/train.json"
test_path = dir_path + "/test.json"

train_df, train_images = load_and_format(train_path)
test_df, test_images = load_and_format(test_path)
x_angle_test = np.array(test_df.inc_angle)   
    
"""
x_train, x_val, y_train, y_val = train_test_split(train_images, to_categorical(train_df['is_iceberg']), 
                                                  random_state=2017, train_size=0.7)

print('Train', x_train.shape, y_train.shape)
print('Validation', x_val.shape, y_val.shape) """

print("reading data")
x_train = pd.read_csv('x_train.csv', header=None)
x_train = x_train.values
y_train = pd.read_csv('y_train.csv', header=None)
y_train = y_train.values
x_val = pd.read_csv('x_val.csv', header=None)
x_val = x_val.values
y_val = pd.read_csv('y_val.csv', header=None)
y_val = y_val.values  

print("reshaping data")
x_train = np.reshape(x_train, (3366, 75, 75, 2))
x_val = np.reshape(x_val, (482, 75, 75, 2))

#0.005 -- 0.9004
#0.01 -- high of 0.9108
weight_decay = 0.02

image_input = Input(shape=(75, 75, 2), name="image")

cnn = BatchNormalization(momentum=0.99)(image_input)

cnn = Conv2D(32, kernel_size=(2,2), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = MaxPooling2D(pool_size=(3,3))(cnn)

cnn = Conv2D(64, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)

cnn = Conv2D(64, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)

cnn = Conv2D(64, kernel_size=(3,3), padding = 'same', kernel_regularizer=l2(weight_decay))(cnn)
cnn = Activation('relu')(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)

cnn = Flatten()(cnn)

cnn = Dense(100, activation='relu', kernel_regularizer=l2(weight_decay))(cnn)

output = Dense(2, activation='softmax')(cnn)


model = Model(inputs=image_input, outputs=output)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()
print("Training")
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20)
model.fit(x_train, y_train, batch_size = 64, validation_data = (x_val, y_val), 
          epochs = 250, shuffle = True, callbacks=[early_stopping])


print("predicting")
test_predictions = model.predict(test_images)

pred_df = test_df[['id']].copy()
pred_df['is_iceberg'] = test_predictions[:,1]
print("creating csv")
pred_df.to_csv('predictions_6.csv', index = False)

Using TensorFlow backend.


reading data
reshaping data
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           (None, 75, 75, 2)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 2)         8         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        288       
_________________________________________________________________
activation_1 (Activation)    (None, 75, 75, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 25, 25, 64) 

3366/3366 [==============================] - 1s - loss: 0.3924 - acc: 0.8904 - val_loss: 0.3862 - val_acc: 0.8714
Epoch 45/250
3366/3366 [==============================] - 1s - loss: 0.4049 - acc: 0.8809 - val_loss: 0.3868 - val_acc: 0.8610
Epoch 46/250
3366/3366 [==============================] - 1s - loss: 0.3896 - acc: 0.8901 - val_loss: 0.3783 - val_acc: 0.8859
Epoch 47/250
3366/3366 [==============================] - 1s - loss: 0.3912 - acc: 0.8892 - val_loss: 0.3893 - val_acc: 0.8900
Epoch 48/250
3366/3366 [==============================] - 1s - loss: 0.3776 - acc: 0.8930 - val_loss: 0.3859 - val_acc: 0.8734
Epoch 49/250
3366/3366 [==============================] - 1s - loss: 0.3639 - acc: 0.8996 - val_loss: 0.3796 - val_acc: 0.8776
Epoch 50/250
3366/3366 [==============================] - 1s - loss: 0.3832 - acc: 0.8913 - val_loss: 0.4003 - val_acc: 0.8776
Epoch 51/250
3366/3366 [==============================] - 1s - loss: 0.3928 - acc: 0.8791 - val_loss: 0.3800 - val_acc: 0.87

3366/3366 [==============================] - 1s - loss: 0.3207 - acc: 0.9177 - val_loss: 0.3641 - val_acc: 0.8859
Epoch 109/250
3366/3366 [==============================] - 1s - loss: 0.3378 - acc: 0.9088 - val_loss: 0.3697 - val_acc: 0.8693
Epoch 110/250
3366/3366 [==============================] - 1s - loss: 0.3400 - acc: 0.9067 - val_loss: 0.3545 - val_acc: 0.8838
Epoch 111/250
3366/3366 [==============================] - 1s - loss: 0.3249 - acc: 0.9130 - val_loss: 0.3734 - val_acc: 0.8672
Epoch 112/250
3366/3366 [==============================] - 1s - loss: 0.3306 - acc: 0.9088 - val_loss: 0.3605 - val_acc: 0.8734
Epoch 113/250
3366/3366 [==============================] - 1s - loss: 0.3150 - acc: 0.9189 - val_loss: 0.3752 - val_acc: 0.8838
Epoch 114/250
3366/3366 [==============================] - 1s - loss: 0.3140 - acc: 0.9189 - val_loss: 0.3633 - val_acc: 0.8880
Epoch 115/250
3366/3366 [==============================] - 1s - loss: 0.3123 - acc: 0.9180 - val_loss: 0.3478 - val_ac